<a href="https://colab.research.google.com/github/ashwanidv100/Data-Analysis-Task/blob/master/HackerEarth_Deep_Learning_Cha_%E2%80%94Auto_tag_Images_of_the_Gala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm 
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/content/dataset/train.csv')
test = pd.read_csv('/content/dataset/test.csv')
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [ ]:
Class_map = {'Food':0, 'Attire':1, 'Decorationandsignage':2, 'misc':3}
inverse_map = {0:'Food', 1:'Attire', 2:'Decorationandsignage', 3:'misc'}
train['Class'] = train['Class'].map(Class_map)

In [ ]:
train['Class']

0       0
1       3
2       1
3       0
4       1
       ..
5978    0
5979    1
5980    0
5981    0
5982    1
Name: Class, Length: 5983, dtype: int64

In [ ]:
train_img = []
train_label = []
j=0
path = '/content/dataset/Train Images'
for i in tqdm(train['Image']):
  final_path = os.path.join(path,i)
  img = cv2.imread(final_path)
  img = cv2.resize(img,(150,150))
  img = img.astype('float32')
  train_img.append(img)
  train_label.append(train['Class'][j])
  j = j+1

100%|██████████| 5983/5983 [00:02<00:00, 2447.64it/s]


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator( featurewise_center=False,  
                              samplewise_center=False, 
                              featurewise_std_normalization=False,  
                              samplewise_std_normalization=False, 
                              zca_whitening=False,  
                              rotation_range=10,  
                              zoom_range = 0.1,
                              width_shift_range=0.2, 
                              height_shift_range=0.2,  
                              horizontal_flip=True,
                              vertical_flip=False )

datagen.fit(train_img)

In [ ]:
test_img = []
path = '/content/dataset/Test Images'
for i in tqdm(test['Image']):
  final_path = os.path.join(path,i)
  img = cv2.imread(final_path)
  img = cv2.resize(img, (150,150))
  img = img.astype('float32')
  test_img.append(img)

100%|██████████| 3219/3219 [00:00<00:00, 3770.48it/s]


In [ ]:
train_img = np.array(train_img)
test_img = np.array(test_img)
train_label = np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(5983, 150, 150, 3)
(3219, 150, 150, 3)
(5983,)


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [ ]:
base_model = VGG16(include_top = False, weights = 'imagenet', input_shape= (150,150,3), pooling = 'avg')

model = Sequential()
model.add(base_model)


model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop

base_model.trainable = False

reduce_learning_rate = ReduceLROnPlateau( monitor='loss',
                                          factor=0.1,
                                          patience=2,
                                          cooldown=2,
                                          min_lr=0.00001,
                                          verbose=1 )

callbacks = [reduce_learning_rate]

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit_generator(datagen.flow(train_img, to_categorical(train_label,4),batch_size= 32), epochs = 30, callbacks = callbacks )



Epoch 1/40
187/187 [==============================] - 30s 161ms/step - loss: 1.6260 - acc: 0.6398
Epoch 2/40
187/187 [==============================] - 30s 160ms/step - loss: 0.7539 - acc: 0.7256
Epoch 3/40
187/187 [==============================] - 30s 162ms/step - loss: 0.5911 - acc: 0.7789
Epoch 4/40
187/187 [==============================] - 31s 165ms/step - loss: 0.5237 - acc: 0.8061
Epoch 5/40
187/187 [==============================] - 30s 159ms/step - loss: 0.4640 - acc: 0.8258
Epoch 6/40
187/187 [==============================] - 30s 159ms/step - loss: 0.4253 - acc: 0.8345
Epoch 7/40
187/187 [==============================] - 31s 166ms/step - loss: 0.3903 - acc: 0.8521
Epoch 8/40
187/187 [==============================] - 30s 161ms/step - loss: 0.3652 - acc: 0.8591
Epoch 9/40
187/187 [==============================] - 30s 162ms/step - loss: 0.3331 - acc: 0.8785
Epoch 10/40
187/187 [==============================] - 31s 164ms/step - loss: 0.3218 - acc: 0.8748
Epoch 11/40
187/187

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 1028      
Total params: 14,847,044
Trainable params: 132,356
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
labels = model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })
submission.head(10)
submission.to_csv('submission.csv', index=False)

[[9.6524018e-01 2.9364601e-02 2.9820422e-04 5.0969585e-03]
 [5.7361704e-01 1.0449989e-05 4.1876619e-06 4.2636830e-01]
 [9.9993634e-01 1.5673559e-18 2.1077989e-16 6.3607957e-05]
 [9.9985528e-01 1.7374344e-13 6.5630112e-14 1.4465187e-04]]
['Food', 'Food', 'Food']
